In [169]:
import numpy as np
import pandas as pd

In [170]:
cardio_data = pd.read_csv('cardio_data.csv', sep = ';')
cardio_data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [171]:
# changing column names
cardio_data.rename(columns = {'ap_hi': 'blood_pressure_max', 'ap_lo': 'blood_pressure_min', 'cholesterol': 'cholesterol_level', 'gluc': 'glucose_level', 'smoke': 'smoking_status', 'alco': 'alcohol_status', 'active': 'activity_level', 'cardio': 'disease'}, inplace = True)
# casting datatypes
cardio_data['height'] = cardio_data['height'].astype('float')
cardio_data['gender'] = cardio_data['gender'].astype('category')
cardio_data['cholesterol_level'] = cardio_data['cholesterol_level'].astype('category')
cardio_data['glucose_level'] = cardio_data['glucose_level'].astype('category')
cardio_data['smoking_status'] = cardio_data['smoking_status'].astype('category')
cardio_data['alcohol_status'] = cardio_data['alcohol_status'].astype('category')
cardio_data['activity_level'] = cardio_data['activity_level'].astype('category')
cardio_data['disease'] = cardio_data['disease'].astype('category')
# converting age from days to years
cardio_data['age'] = cardio_data['age'] // 365
# calculating bmi
cardio_data['bmi'] = cardio_data['weight'] / (cardio_data['height'] / 100)**2

In [168]:
# changing sign of negative blood pressure values
cardio_data['blood_pressure_min'] = cardio_data['blood_pressure_min'].map(lambda x: -1 * x if x < 0 else x)
cardio_data['blood_pressure_max'] = cardio_data['blood_pressure_max'].map(lambda x: -1 * x if x < 0 else x)
# calculating blood pressure difference
cardio_data['blood_pressure_diff'] = cardio_data['blood_pressure_max'] - cardio_data['blood_pressure_min']
# dropping rows with diastolic blood pressure > systolic blood pressure
cardio_data = cardio_data.loc[cardio_data['blood_pressure_diff'] > 0]
# dropping rows with unreasonable bmi values
cardio_data = cardio_data.loc[(cardio_data['bmi'] > 17.5) | (cardio_data['bmi'] < 50)]

SyntaxError: unexpected EOF while parsing (<ipython-input-168-b821839b6ddc>, line 9)

In [172]:
cardio_data.loc[(cardio_data['bmi'] > 17.5) | (cardio_data['bmi'] < 50)]

,id,age,gender,height,weight,blood_pressure_max,blood_pressure_min,cholesterol_level,glucose_level,smoking_status,alcohol_status,activity_level,disease,bmi
0,0,50,2,168.0,62.0,110,80,1,1,0,0,1,0,21.967120
1,1,55,1,156.0,85.0,140,90,3,1,0,0,1,1,34.927679
2,2,51,1,165.0,64.0,130,70,3,1,0,0,0,1,23.507805
3,3,48,2,169.0,82.0,150,100,1,1,0,0,1,1,28.710479
4,4,47,1,156.0,56.0,100,60,1,1,0,0,0,0,23.011177
5,8,60,1,151.0,67.0,120,80,2,2,0,0,0,0,29.384676
6,9,60,1,157.0,93.0,130,80,3,1,0,0,1,0,37.729725
7,12,61,2,178.0,95.0,130,90,3,3,0,0,1,1,29.983588
8,13,48,1,158.0,71.0,110,70,1,1,0,0,1,0,28.440955
9,14,54,1,164.0,68.0,110,60,1,1,0,0,0,0,25.282570
